In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

import multiprocess
# with multiprocess.Pool(processes=8) as pool:
#   for i in pool.imap_unordered(f, a):
#     print(i)

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('19')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [116]:
lines, groups = linesplit("""
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
""".strip())

In [132]:
lines, groups = linesplit(raw_input, verbose=True)

30 lines
1 groups
>>>
Blueprint 1: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 or...
Blueprint 2: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 or...
Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 or...
...


In [136]:
ans = []

for bluep_id, line in enumerate(lines):
  a = amap(int, re.findall(r'-?\d+', line))
  id = a[0]
  rob1_1_cost = a[1]
  rob2_1_cost = a[2]
  rob3_1_cost, rob3_2_cost = a[3:5]
  rob4_1_cost, rob4_3_cost = a[5:7]
  
  states = [
    (
     0, 1,
    0, 0,
    0, 0,
    0, 0,
    )
  ]
  # visited = set()
  
  for time in tqdm(range(32)):
    newpareto = set()
    newstates = []
    
    for state in states:
      h1, r1, h2, r2, h3, r3, h4, r4 = state
      
      if (
          h1 >= rob1_1_cost
          and r1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost)
          and h1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost) * 2
      ):
        newstates.append(
          (
            h1 - rob1_1_cost + r1, r1 + 1,
            h2 + r2, r2,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob2_1_cost
          and r2 < rob3_2_cost
          and h2 < rob3_2_cost * 2
      ):
        newstates.append(
          (
            h1 - rob2_1_cost + r1, r1,
            h2 + r2, r2 + 1,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob3_1_cost and h2 >= rob3_2_cost
          and r3 < rob4_3_cost
          and h3 < rob4_3_cost * 2
      ):
        newstates.append(
          (
            h1 - rob3_1_cost + r1, r1,
            h2 - rob3_2_cost + r2, r2,
            h3 + r3, r3 + 1,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob4_1_cost and h3 >= rob4_3_cost:
        newstates.append(
          (
            h1 - rob4_1_cost + r1, r1,
            h2 + r2, r2,
            h3 - rob4_3_cost + r3, r3,
            h4 + r4, r4 + 1
          )
        )
      
      newstates.append(
        (
          h1 + r1, r1,
          h2 + r2, r2,
          h3 + r3, r3,
          h4 + r4, r4
        )
      )
      
      for i in newstates:
        newpareto.add(i)

    newstates = []
    for i in newpareto:
      should_add = True
      for j in newpareto:
        if i == j:
          continue
        found_better = True
        for ii, jj in zip(i, j):
          if jj >= ii:
            pass
          else:
            found_better = False
            break
        if found_better:
          should_add = False
          break
      if should_add:
        newstates.append(i)
      
    states = newstates
    print(time, len(newpareto), len(states))
  
  ans.append(max([i[-2] for i in states]))

  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 3 3
4 5 3
5 7 5
6 10 6
7 12 9
8 20 14
9 27 14
10 28 17
11 36 19
12 39 18
13 45 25
14 66 32
15 75 41
16 102 63
17 151 75
18 170 101
19 222 128
20 318 192
21 516 312
22 834 457
23 1252 766
24 1966 1168
25 2889 1793
26 4233 2544
27 5683 3413
28 7487 4422
29 9464 5594
30 11871 6857
31 14424 8183


  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 20 14
11 33 24
12 50 28
13 70 47
14 110 57
15 152 99
16 220 108
17 267 133
18 327 155
19 402 186
20 486 267
21 696 381
22 967 521
23 1282 711
24 1677 959
25 2175 1206
26 2662 1505
27 3259 1843
28 3938 2214
29 4684 2595
30 5445 2954
31 6152 3315


  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 19 13
11 30 22
12 43 23
13 49 30
14 67 34
15 84 47
16 117 53
17 135 73
18 190 106
19 267 142
20 356 184
21 525 295
22 840 439
23 1256 714
24 2024 1217
25 3131 1896
26 4564 2600
27 5922 3329
28 7354 4048
29 8759 4788
30 10134 5456
31 11442 6281


  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 19 13
11 30 22
12 43 23
13 48 29
14 59 28
15 62 34
16 87 40
17 101 48
18 111 56
19 144 84
20 202 100
21 242 121
22 268 142
23 313 170
24 374 220
25 563 361
26 966 546
27 1420 750


KeyboardInterrupt: 

In [139]:
ans = []

for bluep_id, line in enumerate(lines):
  a = amap(int, re.findall(r'-?\d+', line))
  id = a[0]
  rob1_1_cost = a[1]
  rob2_1_cost = a[2]
  rob3_1_cost, rob3_2_cost = a[3:5]
  rob4_1_cost, rob4_3_cost = a[5:7]
  
  states = [
    (
     0, 1,
    0, 0,
    0, 0,
    0, 0,
    )
  ]
  # visited = set()
  
  for time in tqdm(range(24)):
    newpareto = set()
    newstates = []
    
    for state in states:
      h1, r1, h2, r2, h3, r3, h4, r4 = state
      
      if (
          h1 >= rob1_1_cost
          and r1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost)
          and h1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost) * 2
      ):
        newstates.append(
          (
            h1 - rob1_1_cost + r1, r1 + 1,
            h2 + r2, r2,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob2_1_cost
          and r2 < rob3_2_cost
          and h2 < rob3_2_cost * 2
      ):
        newstates.append(
          (
            h1 - rob2_1_cost + r1, r1,
            h2 + r2, r2 + 1,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob3_1_cost and h2 >= rob3_2_cost
          and r3 < rob4_3_cost
          and h3 < rob4_3_cost * 2
      ):
        newstates.append(
          (
            h1 - rob3_1_cost + r1, r1,
            h2 - rob3_2_cost + r2, r2,
            h3 + r3, r3 + 1,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob4_1_cost and h3 >= rob4_3_cost:
        newstates.append(
          (
            h1 - rob4_1_cost + r1, r1,
            h2 + r2, r2,
            h3 - rob4_3_cost + r3, r3,
            h4 + r4, r4 + 1
          )
        )
      
      newstates.append(
        (
          h1 + r1, r1,
          h2 + r2, r2,
          h3 + r3, r3,
          h4 + r4, r4
        )
      )
      
      for i in newstates:
        newpareto.add(i)

    newstates = []
    for i in newpareto:
      should_add = True
      for j in newpareto:
        if i == j:
          continue
        found_better = True
        for ii, jj in zip(i, j):
          if jj >= ii:
            pass
          else:
            found_better = False
            break
        if found_better:
          should_add = False
          break
      if should_add:
        newstates.append(i)
      
    states = newstates
    print(time, len(newpareto), len(states))
  
  ans.append(max([i[-2] for i in states]))

  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 3 3
4 5 3
5 7 5
6 10 6
7 12 9
8 20 14
9 27 14
10 28 17
11 36 19
12 39 18
13 45 25
14 66 32
15 75 41
16 102 63
17 151 75
18 170 101
19 222 128
20 318 192
21 516 312
22 834 457
23 1252 766


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 20 14
11 33 24
12 50 28
13 70 47
14 110 57
15 152 99
16 220 108
17 267 133
18 327 155
19 402 186
20 486 267
21 696 381
22 967 521
23 1282 711


  0%|          | 0/24 [00:00<?, ?it/s]

...

0 1 1
1 1 1
2 2 2
3 3 2
4 5 3
5 6 5
6 12 8
7 17 9
8 21 7
9 15 5
10 10 6
11 12 7
12 18 12
13 30 16
14 39 24
15 54 31
16 71 41
17 91 55
18 142 93
19 253 146
20 389 231
21 595 362
22 871 537
23 1237 755


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 19 13
11 30 22
12 43 23
13 48 29
14 58 27
15 57 30
16 68 28
17 75 38
18 95 43
19 105 62
20 152 82
21 198 109
22 256 135
23 307 178


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 2 2
4 4 3
5 6 4
6 8 4
7 10 8
8 17 12
9 27 18
10 36 17
11 39 22
12 60 29
13 72 37
14 92 55
15 133 70
16 159 78
17 169 93
18 198 111
19 233 126
20 274 157
21 393 239
22 596 316
23 779 409


In [140]:
ans

[4,
 9,
 3,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 6,
 5,
 8,
 0,
 1,
 0,
 0,
 0,
 13,
 0,
 5,
 5,
 2,
 5,
 14,
 9,
 4,
 9,
 0,
 3]

In [144]:
ans = []

for bluep_id, line in enumerate(lines[:]):
  a = amap(int, re.findall(r'-?\d+', line))
  id = a[0]
  rob1_1_cost = a[1]
  rob2_1_cost = a[2]
  rob3_1_cost, rob3_2_cost = a[3:5]
  rob4_1_cost, rob4_3_cost = a[5:7]
  
  states = [
    (
     0, 1,
    0, 0,
    0, 0,
    0, 0,
    )
  ]
  # visited = set()
  
  for time in tqdm(range(32)):
    newpareto = set()
    newstates = []
    
    for state in states:
      h1, r1, h2, r2, h3, r3, h4, r4 = state
      
      if (
          h1 >= rob1_1_cost
          and r1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost)
          and h1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost) * 2
      ):
        newstates.append(
          (
            h1 - rob1_1_cost + r1, r1 + 1,
            h2 + r2, r2,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob2_1_cost
          and r2 < rob3_2_cost
          and h2 < rob3_2_cost * 2
      ):
        newstates.append(
          (
            h1 - rob2_1_cost + r1, r1,
            h2 + r2, r2 + 1,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob3_1_cost and h2 >= rob3_2_cost
          and r3 < rob4_3_cost
          and h3 < rob4_3_cost * 2
      ):
        newstates.append(
          (
            h1 - rob3_1_cost + r1, r1,
            h2 - rob3_2_cost + r2, r2,
            h3 + r3, r3 + 1,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob4_1_cost and h3 >= rob4_3_cost:
        newstates.append(
          (
            h1 - rob4_1_cost + r1, r1,
            h2 + r2, r2,
            h3 - rob4_3_cost + r3, r3,
            h4 + r4, r4 + 1
          )
        )
      
      newstates.append(
        (
          h1 + r1, r1,
          h2 + r2, r2,
          h3 + r3, r3,
          h4 + r4, r4
        )
      )
      
      for i in newstates:
        newpareto.add(i)

    newstates = []
    for i in newpareto:
      should_add = True
      for j in newpareto:
        if i == j:
          continue
        found_better = True
        for ii, jj in zip(i, j):
          if jj >= ii:
            pass
          else:
            found_better = False
            break
        if found_better:
          should_add = False
          break
      if should_add:
        newstates.append(i)
      
    states = newstates
    print(time, len(newpareto), len(states))
  
  ans.append(max([i[-2] for i in states]))

  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 3 3
4 5 3
5 7 5
6 10 6
7 12 9
8 20 14
9 27 14
10 28 17
11 36 19
12 39 18
13 45 25
14 66 32
15 75 41
16 102 63
17 151 75
18 170 101
19 222 128
20 318 192
21 516 312
22 834 457
23 1252 766
24 1966 1168
25 2889 1793
26 4233 2544
27 5683 3413
28 7487 4422
29 9464 5594
30 11871 6857
31 14424 8183


  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 20 14
11 33 24
12 50 28
13 70 47
14 110 57
15 152 99
16 220 108
17 267 133
18 327 155
19 402 186
20 486 267
21 696 381
22 967 521
23 1282 711
24 1677 959
25 2175 1206
26 2662 1505
27 3259 1843
28 3938 2214
29 4684 2595
30 5445 2954
31 6152 3315


  0%|          | 0/32 [00:00<?, ?it/s]

...

0 1 1
1 1 1
2 2 2
3 3 2
4 5 3
5 6 5
6 12 8
7 17 9
8 21 7
9 15 5
10 10 6
11 12 7
12 18 12
13 30 16
14 39 24
15 54 31
16 71 41
17 91 55
18 142 93
19 253 146
20 389 231
21 595 362
22 871 537
23 1237 755
24 1680 1002
25 2164 1276
26 2729 1592
27 3368 1937
28 4055 2312
29 4817 2700
30 5606 3125
31 6446 3557


  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 19 13
11 30 22
12 43 23
13 48 29
14 58 27
15 57 30
16 68 28
17 75 38
18 95 43
19 105 62
20 152 82
21 198 109
22 256 135
23 307 178
24 389 208
25 523 332
26 908 512
27 1349 713
28 1907 1100
29 2860 1608
30 4068 2329
31 5631 3299


  0%|          | 0/32 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 2 2
4 4 3
5 6 4
6 8 4
7 10 8
8 17 12
9 27 18
10 36 17
11 39 22
12 60 29
13 72 37
14 92 55
15 133 70
16 159 78
17 169 93
18 198 111
19 233 126
20 274 157
21 393 239
22 596 316
23 779 409
24 1032 596
25 1458 765
26 1789 1049
27 2407 1281
28 2874 1676
29 3722 1993
30 4372 2480
31 5418 2907


In [143]:
ans

[11]

In [138]:
submit2(product(ans))

Submitting 88160...


True

In [141]:
lines

['Blueprint 1: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 15 clay. Each geode robot costs 3 ore and 9 obsidian.',
 'Blueprint 2: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 5 clay. Each geode robot costs 3 ore and 7 obsidian.',
 'Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 11 clay. Each geode robot costs 2 ore and 7 obsidian.',
 'Blueprint 4: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 14 clay. Each geode robot costs 2 ore and 16 obsidian.',
 'Blueprint 5: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 8 clay. Each geode robot costs 2 ore and 18 obsidian.',
 'Blueprint 6: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 14 clay. Each geode robot costs 4 ore and 19 obsidian.',
 'Blueprint 7: 

In [133]:
ans = []

for bluep_id, line in enumerate(lines):
  a = amap(int, re.findall(r'-?\d+', line))
  id = a[0]
  rob1_1_cost = a[1]
  rob2_1_cost = a[2]
  rob3_1_cost, rob3_2_cost = a[3:5]
  rob4_1_cost, rob4_3_cost = a[5:7]
  
  states = [
    (
     0, 1,
    0, 0,
    0, 0,
    0, 0,
    )
  ]
  # visited = set()
  
  for time in tqdm(range(24)):
    newpareto = set()
    newstates = []
    
    for state in states:
      h1, r1, h2, r2, h3, r3, h4, r4 = state
      
      if (
          h1 >= rob1_1_cost
          and r1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost)
          and h1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost) * 2
      ):
        newstates.append(
          (
            h1 - rob1_1_cost + r1, r1 + 1,
            h2 + r2, r2,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob2_1_cost
          and r2 < rob3_2_cost
          and h2 < rob3_2_cost * 2
      ):
        newstates.append(
          (
            h1 - rob2_1_cost + r1, r1,
            h2 + r2, r2 + 1,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob3_1_cost and h2 >= rob3_2_cost and (r3 < rob4_3_cost):
        newstates.append(
          (
            h1 - rob3_1_cost + r1, r1,
            h2 - rob3_2_cost + r2, r2,
            h3 + r3, r3 + 1,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob4_1_cost and h3 >= rob4_3_cost:
        newstates.append(
          (
            h1 - rob4_1_cost + r1, r1,
            h2 + r2, r2,
            h3 - rob4_3_cost + r3, r3,
            h4 + r4, r4 + 1
          )
        )
      
      newstates.append(
        (
          h1 + r1, r1,
          h2 + r2, r2,
          h3 + r3, r3,
          h4 + r4, r4
        )
      )
      
      # for i in newstates:
      #   should_add = True
      #   for j in newpareto:
      #     found_better = True
      #     for ii, jj in zip(i, j):
      #       if jj >= ii:
      #         pass
      #       else:
      #         found_better = False
      #         break
      #     if found_better:
      #       should_add = False
      #       break
      #   if should_add:
      #     newpareto.append(i)
      for i in newstates:
        newpareto.add(i)

    newstates = []
    for i in newpareto:
      should_add = True
      for j in newpareto:
        if i == j:
          continue
        found_better = True
        for ii, jj in zip(i, j):
          if jj >= ii:
            pass
          else:
            found_better = False
            break
        if found_better:
          should_add = False
          break
      if should_add:
        newstates.append(i)
      
    states = newstates
    print(time, len(newpareto), len(states))
  
  ans.append(max([i[-2] for i in states]) * (bluep_id + 1))

  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 3 3
4 5 3
5 7 5
6 10 6
7 12 9
8 20 14
9 27 14
10 28 17
11 36 19
12 39 18
13 45 25
14 66 32
15 75 41
16 102 63
17 151 75
18 170 101
19 222 128
20 318 192
21 516 312
22 837 457
23 1326 777


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 20 14
11 33 24
12 50 28
13 70 47
14 110 57
15 152 99
16 220 108
17 267 133
18 327 155
19 402 186
20 490 267
21 740 390
22 1106 572
23 1605 823


  0%|          | 0/24 [00:00<?, ?it/s]

...

0 1 1
1 1 1
2 2 2
3 3 2
4 5 3
5 6 5
6 12 8
7 17 9
8 21 7
9 15 5
10 10 6
11 12 7
12 18 12
13 30 16
14 39 24
15 54 31
16 71 41
17 91 55
18 142 93
19 253 146
20 407 236
21 667 383
22 1109 624
23 1781 1008


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 1 1
4 3 3
5 5 3
6 5 3
7 7 5
8 10 6
9 13 10
10 19 13
11 30 22
12 43 23
13 48 29
14 58 27
15 57 30
16 68 28
17 75 38
18 95 43
19 105 62
20 152 82
21 198 109
22 256 135
23 307 178


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 1 1
3 2 2
4 4 3
5 6 4
6 8 4
7 10 8
8 17 12
9 27 18
10 36 17
11 39 22
12 60 29
13 72 37
14 92 55
15 133 70
16 159 78
17 169 93
18 198 111
19 233 126
20 274 157
21 393 239
22 596 316
23 798 416


In [134]:
submit1(sum(ans))

Submitting 1962...


True

In [125]:
for i in range(8):
  a = [j[i] for j in states]
  print(i, min(a), max(a))

0 1 53
1 1 4
2 0 119
3 0 14
4 0 23
5 0 6
6 0 5
7 0 2


In [127]:
len(states)

8013

In [126]:
sorted(states[:100])

[(2, 2, 14, 7, 8, 5, 3, 2),
 (2, 2, 34, 8, 10, 2, 4, 1),
 (2, 2, 72, 12, 4, 3, 0, 0),
 (3, 2, 36, 8, 4, 4, 1, 2),
 (3, 4, 79, 11, 4, 2, 0, 0),
 (4, 3, 18, 8, 12, 5, 0, 1),
 (4, 3, 43, 10, 10, 4, 0, 0),
 (4, 3, 54, 10, 14, 3, 0, 0),
 (5, 2, 29, 8, 14, 4, 3, 1),
 (5, 3, 36, 8, 16, 4, 0, 0),
 (5, 4, 30, 9, 18, 4, 0, 0),
 (5, 4, 32, 9, 16, 4, 0, 0),
 (5, 4, 60, 9, 8, 2, 2, 1),
 (6, 3, 31, 9, 9, 4, 2, 1),
 (6, 3, 53, 11, 12, 3, 0, 0),
 (6, 3, 62, 11, 3, 3, 0, 0),
 (7, 3, 45, 10, 10, 3, 1, 1),
 (7, 3, 52, 10, 4, 3, 0, 1),
 (7, 4, 29, 8, 10, 4, 0, 1),
 (8, 2, 98, 12, 5, 1, 0, 0),
 (8, 3, 12, 8, 9, 5, 1, 1),
 (8, 3, 19, 9, 10, 5, 0, 0),
 (8, 3, 28, 7, 4, 4, 1, 2),
 (8, 4, 12, 8, 17, 5, 0, 0),
 (8, 4, 90, 11, 4, 1, 0, 0),
 (9, 3, 25, 9, 9, 4, 2, 1),
 (9, 3, 48, 10, 4, 3, 0, 1),
 (9, 3, 88, 10, 1, 1, 0, 1),
 (9, 4, 10, 8, 17, 5, 0, 0),
 (10, 3, 24, 8, 12, 4, 2, 1),
 (10, 3, 34, 10, 7, 4, 0, 0),
 (10, 4, 36, 8, 12, 3, 3, 1),
 (11, 3, 23, 3, 10, 2, 4, 1),
 (11, 3, 30, 10, 9, 4, 0, 0),
 (11, 3, 43,

In [108]:
sorted(states)

[(1, 1, 20, 5, 0, 0, 0, 0, 1),
 (2, 1, 4, 3, 0, 1, 0, 0, 2),
 (3, 1, 20, 4, 0, 0, 0, 0, 1),
 (3, 2, 10, 5, 0, 0, 0, 0, 1),
 (5, 1, 18, 3, 0, 0, 0, 0, 1),
 (5, 2, 10, 4, 0, 0, 0, 0, 1),
 (5, 5, 0, 0, 0, 0, 0, 0, 0),
 (6, 3, 3, 3, 0, 0, 0, 0, 1),
 (7, 1, 14, 2, 0, 0, 0, 0, 1),
 (7, 2, 9, 3, 0, 0, 0, 0, 1),
 (7, 4, 0, 1, 0, 0, 0, 0, 1),
 (8, 3, 3, 2, 0, 0, 0, 0, 1),
 (9, 1, 8, 1, 0, 0, 0, 0, 1),
 (9, 2, 7, 2, 0, 0, 0, 0, 1),
 (9, 4, 0, 0, 0, 0, 0, 0, 0),
 (10, 3, 2, 1, 0, 0, 0, 0, 1),
 (11, 2, 4, 1, 0, 0, 0, 0, 1),
 (12, 3, 0, 0, 0, 0, 0, 0, 0),
 (13, 2, 0, 0, 0, 0, 0, 0, 0)]

In [131]:
ans

[9, 24]

In [94]:
max([i[-2] for i in states])

4

In [97]:
states[argmax([i[-2] for i in states])]

(9, 2, 31, 6, 8, 4, 6, 3, 3)

In [27]:
for i in states:
  if i[0:4] == [6, 1, 41, 4]:
    print(i)

In [10]:
for bluep_id, line in enumerate(lines):
  a = amap(int, re.findall(r'-?\d+', line))
  id = a[0]
  rob1_1_cost = a[1]
  rob2_1_cost = a[2]
  rob3_1_cost, rob3_2_cost = a[3:5]
  rob4_1_cost, rob4_3_cost = a[5:7]
  
  hm1 = rob1_1_cost + rob2_1_cost + rob3_1_cost + rob4_1_cost
  hm2 = rob3_2_cost
  hm3 = rob4_3_cost
  
  rm1 = max(rob1_1_cost, rob2_1_cost, rob3_1_cost, rob4_1_cost)
  rm2 = rob3_2_cost
  rm3 = rob4_3_cost
  rm4 = 20
  
  print(hm1, hm2, hm3, rm1, rm2, rm3, rm4)
  print(hm1* hm2* hm3* rm1* rm2* rm3* rm4)
  
  dp = np.full((hm1, rm1, hm2, rm2, ), -100)

11 14 7 4 14 7 20
8451520
11 8 12 3 8 12 20
6082560


In [6]:
len(states), len(newstates)

(2220, 31)

In [58]:
len(states)

249

In [45]:
len(states)

42937355

In [19]:
20 * 20 * 20 * 20 * 20 * 20 * 20

1280000000

In [ ]:
count = 0
a = []
s = set()
d = collections.defaultdict()

for line in lines:
  if m := re.findall(r'^$', line):
    pass
  elif m := re.findall(r'^$', line):
    pass
  else:
    raise Exception()